In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc

from scipy.sparse import csr_matrix
from scipy.io import mmread,mmwrite

### txt

In [12]:
adata_raw = pd.read_csv("../Data2Provide/Dataset21_GSE100501/"+"GSM2685238_mRNA_2_PBMCs_matrix.txt",sep = "\t").T
adata_ADT = pd.read_csv("../Data2Provide/Dataset21_GSE100501/"+"GSM2685243_protein_2_PBMCs_matrix.txt",sep = "\t").T

In [17]:
adata_raw = ad.AnnData(adata_raw)
adata_raw.obsm['protein_expression'] = adata_ADT

### csv

In [25]:
adata_raw = sc.read_csv("../Data2Provide/Dataset1_GSE100866/"+"GSE100866_CBMC_8K_13AB_10X-RNA_umi.csv").T
adata_ADT = sc.read_csv("../Data2Provide/Dataset1_GSE100866/"+"GSE100866_CBMC_8K_13AB_10X-ADT_umi.csv").T
adata_raw.obsm['protein_expression'] = adata_ADT.to_df()

### barcode.tsv/feature.tsv/matrix.mtx

In [6]:
RNA_path = '../Data2Provide/dataset13_GSE144744/RNA_counts/'
ADT_path = '../Data2Provide/dataset13_GSE144744/ADT_counts/'

In [7]:
cell_names = pd.read_csv(RNA_path+'barcodes.tsv', sep = '\t', header=None, index_col=None)
# cell_names = pd.read_csv(RNA_path+'barcode.csv', index_col=None)
cell_names.columns =  ['cell_ids'] 
cell_names = cell_names['barcode']

X = csr_matrix(mmread(RNA_path+'matrix.mtx').T)

gene_names = pd.read_csv(RNA_path+'genes.tsv', sep = '\t',  header=None, index_col=None) 
# gene_names = pd.read_csv(RNA_path+'genes.csv', index_col=None) 
gene_names = gene_names['genes']
gene_names = pd.DataFrame(gene_names)
gene_names.columns =  ['gene_ids'] 

adata_raw = ad.AnnData(X, obs=pd.DataFrame(index=cell_names.cell_ids), var=pd.DataFrame(index = gene_names.gene_ids))

In [15]:
## ADT.csv
adata_ADT = sc.read_csv(ADT_path+"ADT.csv",first_column_names=True).T
adata_raw.obsm['protein_expression'] = adata_ADT.to_df()

## ADT_matrix.mtx
X = csr_matrix(mmread(ADT_path + 'adt_matrix.mtx').T)
protein_names = pd.read_csv(ADT_path+'proteins.csv', index_col=None) 
protein_names = protein_names['proteins']
protein_names = pd.DataFrame(protein_names)
protein_names.columns = ['protein_ids']
adata_ADT = ad.AnnData(X, obs=pd.DataFrame(index=cell_names.cell_ids), var=pd.DataFrame(index = protein_names.protein_ids))
adata_raw.obsm['protein_expression'] = adata_ADT.to_df()

### h5ad

In [85]:
adata_total = sc.read_h5ad('../Data2Provide/dataset5_GSE194122/cite_gex_processed_training.h5ad')

In [86]:
count = pd.DataFrame(adata_total.X)
adata_raw = ad.AnnData(count, obs=pd.DataFrame(index=adata_total.obs_names), var=pd.DataFrame(index=adata_total.var_names))

In [88]:
protein_count = pd.DataFrame(adata_total.obsm['protein_expression'])
adata_ADT = ad.AnnData(protein_count, obs=pd.DataFrame(index=adata_total.obs_names), var=pd.DataFrame(index = adata_total.uns['protein_names']))


In [90]:
adata_raw.obsm['protein_expression'] = adata_ADT.to_df()

## Save Data

In [119]:
adata_raw.write("../Data/adata_raw.h5ad")